In [8]:
import os
import re
import json
import random
import sys
import pickle
import datetime
import asyncio
import nest_asyncio
import sympy as sp
import gurobipy as gp

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from tqdm.auto import tqdm
from colorama import Fore, Style
from pydantic import BaseModel
from typing import List
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.llms.lmstudio import LMStudio

sys.path.append('../')
from utils import *

nest_asyncio.apply()

In [9]:
DATA_DIR = '../data'
DATASET_NAME = 'LPWP' 
OUTPUT_DIR = '../output'  

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'LPWP.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
# demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in qa_pairs]
answers = [a for _, a in qa_pairs]

2024-09-19 13:15:12.432 | DEBUG    | utils:read_txt_file:15 - Reading file: ../data\LPWP\LPWP.txt
2024-09-19 13:15:12.433 | DEBUG    | utils:read_txt_file:17 - File read successfully: ../data\LPWP\LPWP.txt
2024-09-19 13:15:12.434 | INFO     | utils:get_nl4opt_qas:36 - Number of questions: 288
2024-09-19 13:15:12.434 | INFO     | utils:get_nl4opt_qas:37 - Number of answers: 288


In [10]:
llm = LMStudio(
    model_name="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    base_url="http://localhost:1234/v1",
    temperature=0.0,
)

In [11]:
class Code(BaseModel):
    # reasoning: List[str]
    code: str     

In [12]:
prompt_template_str = """You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into a piece of code.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:\n"""

example = """
QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEGER)
m.setObjective(10 * x + 15 * y, gp.GRB.MAXIMIZE)
m.addConstr(46 * x + 13 * y <= 1345)
m.addConstr(43 * x + 4 * y <= 346)
m.addConstr(56 * x + 45 * y <= 1643)

QUESTION:
Ben is growing apples and pears on his orchard. He has 50 acres available on which he must grow a minimum of 5 acres of apples and a minimum of 10 acres of pears to meet demands. The profit per apple is $2 and the profit per pear is $4. He prefers to grow more pears than apples but limitations in his workforce allow him to grow at most twice the amount of pears as apples. How many of each fruit should Ben grow in order to maximize his profit? What is that profit?
CODE:
x = m.addVar(name="apples", vtype=gp.GRB.INTEGER)
y = m.addVar(name="pears", vtype=gp.GRB.INTEGER)
m.setObjective(2 * x + 4 * y, gp.GRB.MAXIMIZE)
m.addConstr(x + y <= 50)
m.addConstr(x >= 5)
m.addConstr(y >= 10)
m.addConstr(y <= 2 * x)

Follow the above steps and try to step by step generate the code for the following question.
"""

prompt_template_str = prompt_template_str + example + "\nQUESTIOM: {q}"
print(prompt_template_str)

You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into a piece of code.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:

QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEG

In [13]:
program = LLMTextCompletionProgram.from_defaults(
    output_cls=Code,
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=False,
)

In [57]:
codes = {}
for i in tqdm(range(len(questions))):
    try:
        codes[i] = program(q=questions[i])
    except Exception as e:
        # loguru.logger.error(f"Error for question {i}: {e}")
        codes[i] = f"{e}"
        continue

100%|██████████| 288/288 [13:32<00:00,  2.82s/it]


In [58]:
def parse_value_error_message(text: str) -> str:
    # Regular expression to capture everything after "CODE:"
    pattern = r'"code":\s*"([^"]*)"'

    # Extracting the code after "CODE:"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        code = match.group(1).strip()  # Extract the captured group and strip extra spaces
        return code
    else:
        return None

In [59]:
while True:
    error_idx = []
    for k, v in codes.items():
        if isinstance(v, str):
            if v.startswith("Could not extract json string from output:"):
                codes[k] = parse_value_error_message(v)
            elif v.startswith("1 validation error for Code"):
                error_idx.append(k)
    
    print(error_idx)
    if len(error_idx) == 0:
        break

    for ei in tqdm(error_idx):
        try:
            codes[ei] = program(q=questions[ei])
        except Exception as e:
            # loguru.logger.error(f"Error for question {i}: {e}")
            codes[ei] = f"{e}"
            continue

loguru.logger.info("All questions have been answered")

[2, 4, 7, 42, 43, 46, 55, 68, 70, 78, 104, 108, 127, 134, 140, 144, 158, 194, 225, 246, 259, 270, 273, 281]


100%|██████████| 24/24 [01:05<00:00,  2.72s/it]


[55, 108, 259]


100%|██████████| 3/3 [00:06<00:00,  2.08s/it]


[108]


100%|██████████| 1/1 [00:02<00:00,  2.76s/it]
2024-09-19 14:24:37.588 | INFO     | __main__:<module>:22 - All questions have been answered


[]


In [61]:
codes

{0: Code(code='x, y = m.addVar(name="sleigh", vtype=gp.GRB.INTEGER), m.addVar(name="truck", vtype=gp.GRB.INTEGER)\nm.setObjective(1 * x + 2 * y, gp.GRB.MAXIMIZE)\nm.addConstr(x <= 5 * y)\nm.addConstr(y <= 10 * x)\nm.addConstr(-1000*x + 800*y >= -28000)'),
 1: Code(code='x = m.addVar(name="color printers", vtype=gp.GRB.INTEGER)\ny = m.addVar(name="black and white printers", vtype=gp.GRB.INTEGER)\nm.setObjective(200 * x + 70 * y, gp.GRB.MAXIMIZE)\nm.addConstr(x <= 20)\nm.addConstr(y <= 30)\nm.addConstr(x + y <= 35)'),
 2: "x = m.addVar(name='fertilizer A', vtype=gp.GRB.INTEGER)\\ny = m.addVar(name='fertilizer B', vtype=gp.GRB.INTEGER)\\nm.setObjective(5 * x + 9 * y, gp.GRB.MINIMIZE)\\nm.addConstr(13 * x + 8 * y >= 220)\\nm.addConstr(5 * x + 14 * y >= 160)\\nm.addConstr(6 * x + 6 * y <= 350)",
 3: Code(code='x = m.addVar(name="Pill 1", vtype=gp.GRB.INTEGER)\ny = m.addVar(name="Pill 2", vtype=gp.GRB.INTEGER)\nm.setObjective(0.3 * x + 0.1 * y, gp.GRB.MINIMIZE)\nm.addConstr(0.2 * x + 0.6 * y

In [62]:
code_strs = [codes[i].code if isinstance(codes[i], Code) else codes[i] for i in range(len(codes)) ]

In [63]:
len(code_strs)

288

In [64]:
filename = 'e2e_codegen_llama3.1_8B_lpwp_gurobi_' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(codes, f)

In [65]:
prefix = """
import gurobipy as gp
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = gp.Model(env=env)
"""
                
suffix = """
m.optimize()
"""

def complement_code(code: str) -> float:
    return prefix + code + suffix

In [66]:
def clean_code(code: str) -> str:
    temp_code = code
    # Split the code into lines
    pattern = r'\)([a-zA-Z])'
    temp_code = re.sub(pattern, r')\n\1', temp_code)

    cleand_code = []
    for line in temp_code.split('\n'):
        line = line.strip()
        # Replace > < to >= <=
        if line.startswith('m.addConstr') and not re.findall(r'<=|>=', line):
            # print("Not found")
            line = re.sub(r'<', r'<=', line)
            line = re.sub(r'>', r'>=', line)
        # Remove all comments and suffix and prefix terms
        if not line.startswith('```') and not line.startswith('#'):
            cleand_code.append(line)
        else:
            continue
        # Don't support bool expression '=='
        if re.findall(r'==', line):
            cleand_code.remove(line)
        
    cleand_code = '\n'.join(cleand_code)

    # Remove all '{' and '}'
    cleand_code = cleand_code.replace('{', '').replace('}', '')
    return cleand_code

In [67]:
def execute_code(code: str) -> float:
    ex_locals = {}
    exec(code, None, ex_locals)
    
    try:
        return ex_locals["m"].objVal
    except Exception as e:
        # print(e)
        return np.inf

In [68]:
def get_variables(code: List[str]) -> List[str]:
    vars = []
    for line in code:
        if re.findall(r'addVar', line):
            pattern = r'(\w+)\s*='
            matches = re.findall(pattern, line)
            vars.append(matches[0])
    return sp.symbols(vars)

In [69]:
def simplify_code(code: str) -> str:
    simplfied_code = []
    for i, line in enumerate(code.split('\n')):
        if line.startswith('m.addConstr') or line.startswith('m.setObjective'):
            if '/' in line:
                obj_pattern = r'm\.setObjective\(([^,]*)'
                constr_pattern = r'm\.addConstr\((.*)\)'
                if re.findall(obj_pattern, line):
                    matches = re.findall(obj_pattern, line)
                    obj = re.search(r'gp\.GRB\.(\w+)', line).group(1)
                    expr = sp.sympify(matches[0])
                    simplfied_code.append(f"m.setObjective({str(sp.simplify(expr))}, gp.GRB.{obj})")
                if re.findall(constr_pattern, line):
                    matches = re.findall(constr_pattern, line)
                    oper = re.search(r'\s*(>=|<=)\s*', matches[0]).group(1)
                    expr = sp.sympify(matches[0])
                    simplified_expr = str(sp.simplify(expr.lhs - expr.rhs))
                    if match := re.search(r'^\((.*?)\)/', simplified_expr):
                        new_constr = f'{match.group(1)} {oper} {str(0)}'
                        simplfied_code.append('m.addConstr(' + new_constr + ')')
            else:
                simplfied_code.append(line)
        else:
            simplfied_code.append(line)
    return '\n'.join(simplfied_code)

In [70]:
pred_answers = []
for i, code_str in enumerate(code_strs):
    try:
        cl_code = clean_code(code_str)
        si_code = simplify_code(cl_code)
        co_code = complement_code(si_code)
        ans = execute_code(co_code)
        loguru.logger.info(f"question {i} obtain answer")
        pred_answers.append(ans)
    except Exception as e:
        loguru.logger.error(f"Error for question {i}: {e}")
        pred_answers.append("Error")

2024-09-19 14:25:42.149 | INFO     | __main__:<module>:8 - question 0 obtain answer
2024-09-19 14:25:42.152 | INFO     | __main__:<module>:8 - question 1 obtain answer
2024-09-19 14:25:42.153 | ERROR    | __main__:<module>:11 - Error for question 2: unexpected character after line continuation character (<string>, line 7)
2024-09-19 14:25:42.155 | INFO     | __main__:<module>:8 - question 3 obtain answer
2024-09-19 14:25:42.332 | INFO     | __main__:<module>:8 - question 4 obtain answer
2024-09-19 14:25:42.343 | INFO     | __main__:<module>:8 - question 5 obtain answer


Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-17


2024-09-19 14:25:42.346 | ERROR    | __main__:<module>:11 - Error for question 6: name 'meat' is not defined
2024-09-19 14:25:42.348 | INFO     | __main__:<module>:8 - question 7 obtain answer


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 3 nonzeros
Model fingerprint: 0x5bd03988
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [4e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+02]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


2024-09-19 14:25:42.366 | INFO     | __main__:<module>:8 - question 8 obtain answer
2024-09-19 14:25:42.368 | ERROR    | __main__:<module>:11 - Error for question 9: expected an indented block after function definition on line 19 (<string>, line 20)
2024-09-19 14:25:42.371 | INFO     | __main__:<module>:8 - question 10 obtain answer
2024-09-19 14:25:42.374 | INFO     | __main__:<module>:8 - question 11 obtain answer
2024-09-19 14:25:42.376 | INFO     | __main__:<module>:8 - question 12 obtain answer
2024-09-19 14:25:42.378 | INFO     | __main__:<module>:8 - question 13 obtain answer
2024-09-19 14:25:42.379 | INFO     | __main__:<module>:8 - question 14 obtain answer


...

2024-09-19 14:25:43.810 | INFO     | __main__:<module>:8 - question 268 obtain answer
2024-09-19 14:25:43.812 | INFO     | __main__:<module>:8 - question 269 obtain answer
2024-09-19 14:25:43.814 | INFO     | __main__:<module>:8 - question 270 obtain answer
2024-09-19 14:25:43.817 | INFO     | __main__:<module>:8 - question 271 obtain answer
2024-09-19 14:25:43.819 | INFO     | __main__:<module>:8 - question 272 obtain answer
2024-09-19 14:25:43.819 | ERROR    | __main__:<module>:11 - Error for question 273: expected string or bytes-like object, got 'NoneType'
2024-09-19 14:25:43.819 | ERROR    | __main__:<module>:11 - Error for question 274: expected string or bytes-like object, got 'NoneType'
2024-09-19 14:25:43.821 | INFO     | __main__:<module>:8 - question 275 obtain answer


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Model fingerprint: 0xaf46b4b3
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [8e-01, 6e+00]
  Objective range  [5e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 3e+03]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


2024-09-19 14:25:43.830 | INFO     | __main__:<module>:8 - question 276 obtain answer
2024-09-19 14:25:43.854 | INFO     | __main__:<module>:8 - question 277 obtain answer
2024-09-19 14:25:43.856 | INFO     | __main__:<module>:8 - question 278 obtain answer
2024-09-19 14:25:43.857 | ERROR    | __main__:<module>:11 - Error for question 279: 'tuple' object has no attribute 'lhs'
2024-09-19 14:25:43.859 | INFO     | __main__:<module>:8 - question 280 obtain answer
2024-09-19 14:25:43.860 | INFO     | __main__:<module>:8 - question 281 obtain answer
2024-09-19 14:25:43.863 | INFO     | __main__:<module>:8 - question 282 obtain answer
2024-09-19 14:25:43.864 | INFO     | __main__:<module>:8 - question 283 obtain answer
2024-09-19 14:25:43.873 | INFO     | __main__:<module>:8 - question 284 obtain answer


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x1fcea47b
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 2e+02]
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Found heuristic solution: objective 7.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 7 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+00, best bound 7.000000000000e+00, gap 0.0000%


2024-09-19 14:25:43.882 | INFO     | __main__:<module>:8 - question 285 obtain answer
2024-09-19 14:25:43.884 | INFO     | __main__:<module>:8 - question 286 obtain answer
2024-09-19 14:25:43.885 | INFO     | __main__:<module>:8 - question 287 obtain answer


In [77]:
def mark(pred, real, error: float) -> List[bool]:    
    correct = []
    for p, r in zip(pred, real):
        if p == 'Error':
            continue
        if float(r) != 0:
            if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < error):
                correct.append(True)
            else:
                correct.append(False)
        else:
            if float(p) < error:
                correct.append(True)
            else:
                correct.append(False)
    return correct

In [79]:
print(f"Accuracy under error {1e-2}: {sum(mark(pred_answers, answers[:35], 1e-2)) / len(answers[:35]) * 100}")
print(f"Accuracy under error {1e-4}: {sum(mark(pred_answers, answers[:35], 1e-6)) / len(answers[:35]) * 100}")
print(f"Accuracy under error {1e-6}: {sum(mark(pred_answers, answers[:35], 1e-6)) / len(answers[:35]) * 100}")

Accuracy under error 0.01: 34.285714285714285
Accuracy under error 0.0001: 34.285714285714285
Accuracy under error 1e-06: 34.285714285714285
